In [7]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from pyproj import Proj, Transformer

In [8]:
# 동 이름 입력
dong = "금호동"

# 네이버 부동산 URL
url = f"https://new.land.naver.com/offices?ms=35.154807,126.9011,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"

In [9]:
# Chrome 드라이버 설정
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get(url)

In [10]:
# 데이터프레임 초기화
columns = ['동', '위도', '경도', '상가종류', '월세', '평수', '층수', '건물번호', '제목', '가격']
df = pd.DataFrame(columns=columns)

In [11]:
# 핀 요소들 찾기
pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")

# 전체 매물 수 계산
total_materials = 0
for pin in pin_elements:
    numbers = re.findall(r'\d+', pin.text)
    if numbers:
        total_materials += int(numbers[0])

print(f"총 매물 수: {total_materials}")

총 매물 수: 7


In [12]:
# 지도 중심 좌표 계산
map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
map_center_x = map_element.size['width'] / 2
map_center_y = map_element.size['height'] / 2

# URL에서 중심 좌표와 줌 레벨 추출
url = driver.current_url
center_lat = float(url[url.find("ms=")+3:url.find(",")])
center_lon = float(url[url.find(",")+1:url.find(",16")])
zoom_level = int(url[url.find(",16")+1:url.find("&a")])

# UTM 변환기 설정
utm_proj = Proj(proj='utm', zone=52, ellps='WGS84')
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(wgs84_proj, utm_proj)

# 중심 좌표를 UTM으로 변환
center_utm_x, center_utm_y = transformer.transform(center_lon, center_lat)

# 줌 레벨에 따른 픽셀당 미터 계산
# 네이버 지도는 줌 레벨 16일 때 1픽셀 = 약 0.6미터
base_meters_per_pixel = 0.6
zoom_factor = 2 ** (16 - zoom_level)
meters_per_pixel = base_meters_per_pixel * zoom_factor

In [13]:
# driver.refresh()

In [18]:
# 핀 클릭 및 데이터 수집
for pin in pin_elements:
    try:
        # 핀 위치 계산
        target_x = float(pin.get_attribute("style").split(" ")[-1].replace("px;", ""))
        target_y = float(pin.get_attribute("style").split(" ")[-3].replace("px;", ""))
        
        # 핀의 UTM 좌표 계산
        delta_x = (target_x - map_center_x) * meters_per_pixel
        delta_y = (map_center_y - target_y) * meters_per_pixel  # y축 방향 반전
        
        pin_utm_x = center_utm_x + delta_x
        pin_utm_y = center_utm_y + delta_y
        
        # UTM을 위경도로 변환
        pin_lon, pin_lat = transformer.transform(pin_utm_x, pin_utm_y, direction='INVERSE')
        
        # 핀 클릭
        pin.click()
        time.sleep(2)  # 로딩 시간 증가
        
        # 매물 목록 로딩
        try:
            body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", body)
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
        except:
            pass
        
        # 매물 목록 가져오기
        items = driver.find_elements(By.CSS_SELECTOR, "div.item")
        
        for i in range(len(items)):
            try:
                # 매물 목록을 다시 가져오기
                current_items = driver.find_elements(By.CSS_SELECTOR, "div.item")
                if i >= len(current_items):
                    continue
                    
                # 매물 상세 정보 클릭
                current_items[i].click()
                time.sleep(2)
                
                # 상세 정보 가져오기
                try:
                    # 상가종류
                    shop_type = driver.find_element(By.CSS_SELECTOR, "h4.info_title").text
                    
                    # 모든 테이블 행을 찾기
                    rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")
                    
                    # 기본값 설정
                    floor = ""
                    area = ""
                    building_no = ""
                    
                    # 각 행에서 정보 추출
                    for row in rows:
                        try:
                            th = row.find_element(By.CSS_SELECTOR, "th").text
                            td = row.find_element(By.CSS_SELECTOR, "td").text
                            
                            if "층" in th:
                                floor = td
                            elif "면적" in th:
                                area = td
                            elif "매물번호" in th:
                                building_no = td
                        except:
                            continue
                except Exception as e:
                    print(f"상세 정보 가져오기 에러: {e}")
                    continue
                
                # 상세 페이지 닫기
                close_btn = driver.find_element(By.CSS_SELECTOR, "button.btn_close[aria-label='상세페이지 닫기']")
                close_btn.click()
                time.sleep(1)
                
            except Exception as e:
                print(f"매물 상세 클릭 에러: {e}")
                continue
        
        # 핀 다시 클릭하여 닫기
        pin.click()
        time.sleep(1)
        
    except Exception as e:
        print(f"핀 클릭 에러: {e}")
        continue

KeyboardInterrupt: 

In [16]:
# 데이터 저장
df.to_csv(f"./data_0414/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data_0414/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)

In [17]:
df

,동,위도,경도,상가종류,월세,평수,층수,건물번호,제목,가격
